<a href="https://colab.research.google.com/github/DNason1999/DS-Unit-1-Build/blob/master/MTG_Analyser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://github.com/DNason1999/txtfilesdataset/blob/master/datasetfiles.zip?raw=true -O dataset

--2019-10-22 21:02:25--  https://github.com/DNason1999/txtfilesdataset/blob/master/datasetfiles.zip?raw=true
Resolving github.com (github.com)... 140.82.118.4
Connecting to github.com (github.com)|140.82.118.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/DNason1999/MtG-MythicChampionship-V-decklists/blob/master/datasetfiles.zip?raw=true [following]
--2019-10-22 21:02:26--  https://github.com/DNason1999/MtG-MythicChampionship-V-decklists/blob/master/datasetfiles.zip?raw=true
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/DNason1999/MtG-MythicChampionship-V-decklists/raw/master/datasetfiles.zip [following]
--2019-10-22 21:02:26--  https://github.com/DNason1999/MtG-MythicChampionship-V-decklists/raw/master/datasetfiles.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubu

In [0]:
!unzip dataset;

Archive:  dataset
  inflating: datasetfiles/Aaron Barich - Golos Fires.txt  
replace datasetfiles/Alexander Hayne - Bant Golos.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: datasetfiles/Alexander Hayne - Bant Golos.txt  
  inflating: datasetfiles/Andrea Mengucci - Bant Food.txt  
  inflating: datasetfiles/Andrew Cuneo - Simic Food.txt  
  inflating: datasetfiles/Ashley Muдoz - Golgari Adventure.txt  
  inflating: datasetfiles/Autumn Burchett - Golos Fires.txt  
  inflating: datasetfiles/Ben Stark - Mardu Knights.txt  
  inflating: datasetfiles/Bertrand Fagnoni - Bant Golos.txt  
  inflating: datasetfiles/Brad Nelson - Bant Golos.txt  
  inflating: datasetfiles/Breanna Lee Tan - Gruul Aggro.txt  
  inflating: datasetfiles/Brian Braun-Duin - Bant Golos.txt  
  inflating: datasetfiles/Caleb Durward - Jund Midrange.txt  
datasetfiles/Carlos Romão - Bant Golos.txt:  mismatching "local" filename (datasetfiles/Carlos Rom├гo - Bant Golos.txt),
         continuing with "central" fil

In [0]:
!wget https://archive.scryfall.com/json/scryfall-default-cards.json

--2019-10-22 21:02:40--  https://archive.scryfall.com/json/scryfall-default-cards.json
Resolving archive.scryfall.com (archive.scryfall.com)... 104.25.224.106, 104.25.223.106, 2606:4700:20::6819:df6a, ...
Connecting to archive.scryfall.com (archive.scryfall.com)|104.25.224.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/json]
Saving to: ‘scryfall-default-cards.json.1’

scryfall-default-ca     [        <=>         ] 148.29M  99.3MB/s    in 1.5s    

2019-10-22 21:02:42 (99.3 MB/s) - ‘scryfall-default-cards.json.1’ saved [155497874]



In [0]:
import pandas as pd
import time

In [0]:
cards = pd.read_json('/content/scryfall-default-cards.json')

In [0]:
important_columns = ['name','color_identity','colors','mana_cost','cmc','power','toughness','rarity','id','type_line']

cards0 = cards[important_columns]
cards0['name'].apply(lambda x : x.strip());

In [0]:
def process_decks(content_path,starting_point,ending_point):
  begin = time.time()
  columns0 = ['qty','name','color_identity','colors','mana_cost','cmc','power','toughness','rarity','type_line']
  y = !ls /content/datasetfiles
  a = []
  b = []
  c = []
  counter = 0
  for deck_name in y[starting_point:ending_point]:
    p = content_path+"/"+deck_name
    p = p.replace("'", "")
    df= pd.read_csv(p, names=['item'], sep='\t')
    deck_data= pd.DataFrame(columns= columns0)
    counter0 = 0
    for label, content in df['item'].items():
      x= content.split(sep= ' ', maxsplit=1)

      cardid = None
      url = 'https://api.scryfall.com/cards/search?q='+x[1]
      url = url.replace(" ","+")

      while(cardid == None):
        try:
          cardid = pd.read_json(url)['data'][0]['id']
        except: time.sleep(1)

      cards_index = cards0.iloc[cards0['id'][cards0['id']==cardid].index[0]]

      c0 = cards_index[columns0]
      c0['qty'] = x[0]
      c0['name']= x[1]

      deck_data = deck_data.append(c0)

      print("Deck Progress:", counter0,'/',len(df['item']))
      counter0 += 1

      time.sleep(0.1)

    x = deck_name.split(sep = '-', maxsplit=1)
    deck_author = x[0]
    deck_name = x[1]

    deck_name = deck_name.split(sep= '.txt', maxsplit=1)

    deck_data.reindex()
    a.append(deck_author)
    b.append(deck_name)
    c.append(deck_data)
    counter += 1
    print("Percent Complete:",int((counter/len(y))*100), "\tTime Elapsed:", begin-time.time(), "\tCounter:",counter)
  d = [a,b,c]
  end = time.time()
  print(begin-end)
  return d


In [0]:
main = pd.DataFrame(columns=['author','name','deck'])

In [0]:
information = process_decks('/content/datasetfiles',0,68);

In [0]:
main['author'] = information[0]
main['name'] = information[1]
main['deck'] = information[2]

In [0]:
def to_list(input_dataframe):
  t = []
  for label,content in input_dataframe.iterrows():
    t.append(content.to_list())
  return t


In [0]:
main0 = main.copy()
main0['deck'] = main0['deck'].apply(lambda x : to_list(x))
main0['name'] = main0['name'].apply(lambda x : str(x[0]))
main0['author'] = main0['author'].apply(lambda x : x.replace("'",""))

In [0]:
for label,content in main0.iterrows():
  if(content[1].count('-') != 0):
    z = content[1].split('-')
    content[1] = z[1]
    content[0] = content[0]+' '+z[0]

In [0]:
main0.head()

,author,name,deck
0,Aaron Barich,Golos Fires,"[[1, Agent of Treachery, [U], [U], {5}{U}{U}, ..."
1,Alexander Hayne,Bant Golos,"[[2, Teferi, Time Raveler, [U, W], [U, W], {1}..."
2,Andrea Mengucci,Bant Food,"[[4, Oko, Thief of Crowns, [G, U], [G, U], {1}..."
3,Andrew Cuneo,Simic Food,"[[4, Oko, Thief of Crowns, [G, U], [G, U], {1}..."
4,Ashley Muдoz,Golgari Adventure,"[[4, Edgewall Innkeeper, [G], [G], {G}, 1.0, 1..."


In [0]:
main0.tail()

,author,name,deck
63,Stephan Schwarz,Selesnya Adventure,"[[4, Faerie Guidemother, [W], [W], {W} // {1}{..."
64,Takeshi Senoo,Golgari Adventure,"[[4, Foulmire Knight, [B], [B], {B} // {2}{B},..."
65,ThВo Moutier,Golos Fires,"[[3, Beanstalk Giant, [G], [G], {6}{G} // {2}{..."
66,William Jensen,Simic Food,"[[4, Oko, Thief of Crowns, [G, U], [G, U], {1}..."
67,Yuuki Ichikawa,Simic Food,"[[4, Nissa, Who Shakes the World, [G], [G], {3..."


In [0]:
main0['author'] = main0['author'].apply(lambda x : x.split()[0]+" "+x.split()[1][0])

In [0]:
a = main0.to_csv(index=False)

In [0]:
from google.colab import files

In [0]:
with open('data.csv', 'w') as f:
  f.write(a)

files.download('data.csv')